In [1]:
# %pip install python-dotenv


In [2]:
import os
import sys
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import sqlalchemy as sq
from dotenv import load_dotenv
from matplotlib import pyplot as plt
from DataService import DataService

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
os.chdir('/tf')
load_dotenv()


2022-11-22 00:23:35.058662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 00:23:35.191059: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


True

In [3]:
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')


In [4]:
dataService = DataService(PGDB, PGUSER, PGPW)
dbConn = dataService.connect()


In [5]:
# Grab large fire gis data for the last 10 years from postgres
query = "SELECT * FROM public.\"lgFireFifty\" WHERE \"YEAR\" > 2009;"
fire = gpd.GeoDataFrame.from_postgis(query, dbConn, geom_col='geom')
fire.head()


,EntryID,FIRE_ID,FIRENAME,YEAR,MONTH,DAY,REP_DATE,OUT_DATE,DECADE,SIZE_HA,CALC_HA,CAUSE,CFS_REF_ID,geom
0,1,HWF278,Birch Complex Fire,2015,7,6,2015-07-06,None,2010-2019,3329.7,3329.9,L,AB-2015-HWF278,"POLYGON ((4932200.196 2611443.456, 4932314.213..."
1,2,HWF280,None,2017,9,1,2017-09-01,None,2010-2019,13628.3,13638.3,U,AB-2017-HWF280,"MULTIPOLYGON (((4834104.255 2898701.409, 48339..."
2,33,LWF116,Cowper Complex Fire,2018,5,22,2018-05-22,None,2010-2019,1456.2,1457.3,L,AB-2018-LWF116,"MULTIPOLYGON (((5036526.749 2375899.118, 50365..."
3,6,HWF291,None,2017,9,24,2017-09-24,None,2010-2019,1520.7,1521.4,U,AB-2017-HWF291,"POLYGON ((4711737.650 2814312.160, 4711570.367..."
4,7,HWF292,None,2017,9,25,2017-09-25,None,2010-2019,448.9,449.1,U,AB-2017-HWF292,"POLYGON ((4711219.479 2814219.324, 4711097.577..."


In [6]:
fire.describe()


,EntryID,YEAR,MONTH,DAY,SIZE_HA,CALC_HA
count,3265.0,3265.0,3265.0,3265.0,3265.0,3265.0
mean,6282.6,2014.5,6.7,16.1,8200.9,8125.4
std,3983.5,2.8,1.1,9.2,32210.4,31947.1
min,1.0,2010.0,2.0,1.0,200.0,36.1
25%,3056.0,2012.0,6.0,7.0,487.0,482.5
50%,5708.0,2015.0,7.0,16.0,1375.7,1370.4
75%,9085.0,2017.0,7.0,25.0,4719.8,4664.7
max,12827.0,2020.0,12.0,31.0,596459.1,596459.1


In [8]:
# Grab fire centroids for the last 10 years from postgres
query = "Select f1.\"EntryID\", f1.geom FROM \"lgFireFiftyCentroids\" as f1 \
            JOIN \"lgFireFifty\" lFF on f1.\"EntryID\" = lFF.\"EntryID\" \
            WHERE \"YEAR\" > 2009;"
fireCentroids = gpd.GeoDataFrame.from_postgis(query, dbConn, geom_col='geom')
fireCentroids.head()


,EntryID,geom
0,1,POINT (4934547.488 2608092.858)
1,2,POINT (4820621.875 2904194.413)
2,33,POINT (5034115.756 2378216.407)
3,6,POINT (4711296.400 2816704.501)
4,7,POINT (4710313.249 2815071.918)


In [9]:
fireCentroids.describe()


,EntryID
count,3265.0
mean,6282.6
std,3983.5
min,1.0
25%,3056.0
50%,5708.0
75%,9085.0
max,12827.0


In [10]:
# Grab Province GIS boundaries from postgres
query = "SELECT * FROM public.\"dbfProvBound\";"
prov = gpd.GeoDataFrame.from_postgis(query, dbConn, geom_col='geom')
prov.head()

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geom
0,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,Newfoundland and Labrador,Terre-Neuve-et-Labrador,N.L.,T.-N.-L.,"MULTIPOLYGON (((8307365.589 2582136.711, 83083..."
1,11,b'Prince Edward Island / \xcele-du-Prince-\xc9...,Prince Edward Island,b'\xcele-du-Prince-\xc9douard',P.E.I.,b'\xce.-P.-\xc9.',"MULTIPOLYGON (((8435711.754 1679935.966, 84358..."
2,12,b'Nova Scotia / Nouvelle-\xc9cosse',Nova Scotia,b'Nouvelle-\xc9cosse',N.S.,b'N.-\xc9.',"MULTIPOLYGON (((8470851.646 1624745.011, 84710..."
3,13,New Brunswick / Nouveau-Brunswick,New Brunswick,Nouveau-Brunswick,N.B.,N.-B.,"MULTIPOLYGON (((8176224.537 1722553.460, 81762..."
4,24,b'Quebec / Qu\xe9bec',Quebec,b'Qu\xe9bec',Que.,Qc,"MULTIPOLYGON (((8399709.494 2261445.703, 84005..."


In [11]:
prov.describe()

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:1784: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/inference.py:387: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/inference.py:388: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/usr/local/lib/python3.8/dist-packages/pandas/i

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geom
count,13,13,13,13,13,13,13
unique,13,13,13,13,13,13,13
top,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,Newfoundland and Labrador,Terre-Neuve-et-Labrador,N.L.,T.-N.-L.,"(POLYGON ((8307365.58857 2582136.71143, 830839..."
freq,1,1,1,1,1,1,1


In [ ]:
# We use province boundaries to get a subset of the fire data based on centroids 
# start with SK
fireSK = gpd.GeoDataFrame.sjoin()